---

## Set Up PostgreSQL (on a Mac)

Install with Homebrew:

```
$ brew update && brew install postgres
```

Or, if already installed,

```
$ brew update && brew upgrade postgres
```

Run `initdb` just once, basically to create the directory structure and such on disk that's needed for creating new databases. Note: The specified path should match the version of Postgres just installed!

```
$ initdb /usr/local/var/postgres9.5 -E utf8
```

To _manually_ start and stop a local Postgres server from running, use

```
$ pg_ctl -D /usr/local/var/postgres -l /usr/local/var/postgres/server.log start
$ pg_ctl -D /usr/local/var/postgres stop -s -m fast
```
        
Or to _automatically_ start a Postgres server (now and) at launch:

```
$ mkdir -p ~/Library/LaunchAgents
$ ln -sfv /usr/local/opt/postgresql/*.plist ~/Library/LaunchAgents
$ launchctl load ~/Library/LaunchAgents/homebrew.mxcl.postgresql.plist
```

Open the system paths file, `/etc/paths`, in a text editor, and move the line `/usr/local/bin` from the bottom of the file to the top (if it wasn't like this already). If you had to make a change to the file, reboot the computer. After rebooting, the command `which psql` command should return `/usr/local/bin/psql`.

Homebrew automatically created a database superuser account with the same login as your current Mac OS account. Let's create a dedicated user named `app` for connecting to and owning the app's database:

```
$ createuser --echo --pwprompt --superuser --createdb app
```

You'll be prompted to create a password — be sure to remember it or save it somewhere!

Homebrew also automatically created a database named `postgres` that may be used to log info for administrative tasks such as creating a user. Let's create a new database for this project:

```
$ createdb --echo --encoding=utf8 --host=<HOST> --port=<PORT> --username=app --owner=app <DBNAME>
```

We're using our `app` user to create the database, and assign it as the db's owner. **TODO:** Choose a `<DBNAME>` for this project!

To access the database through an interactive shell:

```
$ psql --host=<HOST> --port=<PORT> --username=app --dbname=<DBNAME>
```

Lastly, define a `DATABASE_URL` environment variable containing this same information so that programs such as those in `cipy` can access it:

```
$ export DATABASE_URL=postgres://app<PASSWORD>@<HOST>/<DBNAME>
```

---

## Create Citations Table from Templates and DDL

In [1]:
import cipy

In [2]:
citations_ddl = cipy.db.get_ddl('citations')
conn_creds = cipy.db.get_conn_creds(env_var='DATABASE_URL')
pgdb = cipy.db.PostgresDB(conn_creds, citations_ddl)

In [3]:
pgdb.create_table()

In [6]:
pgdb.drop_table(act=False)

(act=False)
b'DROP TABLE IF EXISTS citations'


In [5]:
pgdb.print_table_spec('citations')

name                           pos type                           nullable max_length
-------------------------------------------------------------------------------------
citation_id                      1 bigint                         NO                 
project_id                       2 integer                        NO                 
user_id                          3 integer                        NO                 
insert_ts                        4 timestamp without time zone    NO                 
type_of_work                     5 character varying              YES      25        
title                            6 character varying              YES      250       
secondary_title                  7 character varying              YES      250       
publication_year                 8 smallint                       YES                
publication_month                9 smallint                       YES                
authors                         10 ARRAY              

In [4]:
list(pgdb.run_query('SELECT COUNT(1) FROM citations'))

[{'count': 24020}]